In [2]:
import sys
print(sys.executable)
print(sys.version)

!{sys.executable} -m pip show mediapipe

!python3.10 -m pip install -Uqq mediapipe

/opt/anaconda3/bin/python3.10
3.10.0 (default, Mar  3 2022, 03:54:28) [Clang 12.0.0 ]
Name: mediapipe
Version: 0.10.21
Summary: MediaPipe is the simplest way for researchers and developers to build world-class ML solutions and applications for mobile, edge, cloud and the web.
Home-page: https://github.com/google/mediapipe
Author: The MediaPipe Authors
Author-email: mediapipe@google.com
License: Apache 2.0
Location: /opt/anaconda3/lib/python3.10/site-packages
Requires: absl-py, attrs, flatbuffers, jax, jaxlib, matplotlib, numpy, opencv-contrib-python, protobuf, sentencepiece, sounddevice
Required-by: 


In [13]:
import pandas as pd
import mediapipe as mp
from PIL import Image
from pathlib import Path
from tqdm.notebook import tqdm
import cv2

# Paths to your datasets and images
data_dir = Path('./data')
image_labels_csv = data_dir / 'image_labels.csv'
bodyfat_csv = data_dir / 'bodyfat_dataset.csv'
images_dir = Path('images')  # Adjust this path if your images are elsewhere

# Load CSVs
df_labels = pd.read_csv(image_labels_csv)
df_bodyfat = pd.read_csv(bodyfat_csv)

print(f"Labels df shape: {df_labels.shape}")
print(f"Bodyfat df shape: {df_bodyfat.shape}")

# Setup mediapipe pose detector
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.5)

def crop_upper_body(image_path):
    img = cv2.imread(str(image_path))
    if img is None:
        print(f"Failed to read image {image_path}")
        return None

    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = pose.process(img_rgb)

    if not results.pose_landmarks:
        print(f"No pose detected in {image_path}")
        return None

    try:
        landmarks = results.pose_landmarks.landmark

        left_shoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER]
        right_shoulder = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER]
        left_hip = landmarks[mp_pose.PoseLandmark.LEFT_HIP]
        right_hip = landmarks[mp_pose.PoseLandmark.RIGHT_HIP]

        h, w, _ = img.shape
        xs = [left_shoulder.x * w, right_shoulder.x * w, left_hip.x * w, right_hip.x * w]
        ys = [left_shoulder.y * h, right_shoulder.y * h, left_hip.y * h, right_hip.y * h]

        xmin, xmax = int(min(xs)), int(max(xs))
        ymin, ymax = int(min(ys)), int(max(ys))

        pad_x, pad_y = 20, 20
        xmin = max(xmin - pad_x, 0)
        xmax = min(xmax + pad_x, w)
        ymin = max(ymin - pad_y, 0)
        ymax = min(ymax + pad_y, h)

        cropped = img[ymin:ymax, xmin:xmax]
        cropped_rgb = cv2.cvtColor(cropped, cv2.COLOR_BGR2RGB)
        return Image.fromarray(cropped_rgb)

    except Exception as e:
        print(f"Error cropping {image_path}: {e}")
        return None

# Create folder for cropped images if it doesn't exist
cropped_dir = Path('cropped_images')
cropped_dir.mkdir(exist_ok=True)

cropped_paths = []
failed_images = []

for idx, row in tqdm(df_labels.iterrows(), total=len(df_labels)):
    filename = row['filename']
    img_path = images_dir / filename
    cropped_img = crop_upper_body(img_path)
    if cropped_img is not None:
        save_path = cropped_dir / filename
        cropped_img.save(save_path)
        cropped_paths.append(str(save_path))
    else:
        failed_images.append(filename)
        cropped_paths.append(None)  # To keep alignment with df_labels rows

print(f"Done! Cropped {len([p for p in cropped_paths if p])} images, failed on {len(failed_images)} images.")

# Add cropped image paths to dataframe and save
df_labels['cropped_path'] = cropped_paths
df_labels.to_csv(data_dir / 'image_labels_cropped.csv', index=False)


Labels df shape: (1596, 2)
Bodyfat df shape: (806, 10)


I0000 00:00:1749685857.614809 4260317 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M4


  0%|          | 0/1596 [00:00<?, ?it/s]

No pose detected in images/0_image_1.jpg
No pose detected in images/0_image_2.jpg
No pose detected in images/0_image_3.jpg


W0000 00:00:1749685857.696768 4273306 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1749685857.711918 4273306 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


...

[ WARN:0@539.306] global loadsave.cpp:268 findDecoder imread_('images/762_image_1.jpg'): can't open/read file: check file path/integrity


No pose detected in images/766_image_1.jpg
No pose detected in images/769_image_3.png
No pose detected in images/770_image_1.jpg
No pose detected in images/770_image_2.jpg
No pose detected in images/771_image_1.jpg
No pose detected in images/774_image_1.jpg
Failed to read image images/780_image_1.jpg
No pose detected in images/782_image_3.jpg


[ WARN:0@541.094] global loadsave.cpp:268 findDecoder imread_('images/780_image_1.jpg'): can't open/read file: check file path/integrity


No pose detected in images/784_image_1.jpg
No pose detected in images/785_image_2.jpg
No pose detected in images/785_image_3.jpg
No pose detected in images/792_image_2.jpg
No pose detected in images/792_image_3.jpg
No pose detected in images/796_image_1.jpg
No pose detected in images/801_image_1.jpg
Failed to read image images/802_image_1.jpg
Failed to read image images/803_image_1.jpg
Done! Cropped 1357 images, failed on 239 images.


[ WARN:0@543.420] global loadsave.cpp:268 findDecoder imread_('images/802_image_1.jpg'): can't open/read file: check file path/integrity
[ WARN:0@543.420] global loadsave.cpp:268 findDecoder imread_('images/803_image_1.jpg'): can't open/read file: check file path/integrity
